In [1]:
import numpy as np
import os.path
import math

In [21]:
class patchFamily2D:
    def __init__(self,file_ID,ndof):
        # Knot Vectors
        self.U = []
        # Control Points
        self.CP = []
        # Orders
        self.p = []
        # Number of Shape Functions
        self.nsf = []
        # Connectivity
        self.nel = []
        self.nnp = []
        self.nen = []
        self.INC = []
        self.IEN = []
        self.ID = []
        self.LM = []
        # READ PATCH FAMILY NAMED WITH "file_ID"
        file_exists = os.path.exists(file_ID+'1')
        fNo = 1
        while (file_exists == True):
            f = open(file_ID+str(fNo), "r")
            data = f.readlines()
            # Knot Vector - V
            lenV = int(data[0])
            V_p = np.zeros(lenV)
            dataV = data[1].split(" ")
            for i in range(0,lenV):
                V_p[i] = float(dataV[i])
            # Knot Vector - U
            lenU = int(data[2])
            U_p = np.zeros(lenU)
            dataU = data[3].split(" ")
            for i in range(0,lenU):
                U_p[i] = float(dataU[i])
            self.U.append([])
            self.U[fNo-1].append(U_p)
            self.U[fNo-1].append(V_p)
            # Control Points:
            lenCP = int(data[4])
            CP_p = np.zeros((lenCP,4))
            for i in range(0,lenCP):
                dataCP = data[i+5].split(" ")
                for j in range(0,4):
                    CP_p[i][j] = float(dataCP[j])
            self.CP.append(CP_p)
            # Order:
            p1 = 0
            p2 = 0
            while True:
                if (U_p[p1] == U_p[p1+1]):
                    p1 += 1
                else:
                    break
            while True:
                if (V_p[p2] == V_p[p2+1]):
                    p2 += 1
                else:
                    break  
            self.p.append(np.array([p1,p2]))
            n = len(U_p)-p1-1
            m = len(V_p)-p2-1
            self.nsf.append(np.array([n,m]))
            # Connectivity:
            nel = (n-p1)*(m-p2)
            self.nel.append(nel)
            nnp = n*m
            self.nnp.append(nnp)
            nen = (p1+1)*(p2+1)
            self.nen.append(nen)
            INC = np.zeros((n*m,2)).astype(int)
            IEN = np.zeros((nel,nen)).astype(int)
            A = 0
            B = 0
            for j in range(1,m+1):
                for i in range(1,n+1):
                    INC[A][0] = (i-1)
                    INC[A][1] = (j-1)
                    if (i >= p1+1 and j >= p2+1):
                        for k in range(0,p2+1):
                            for l in range(0,p1+1):
                                C = int(k*(p1+1)+l)
                                IEN[B][C] = ((A-k*n-l))
                        B += 1
                    A += 1
            self.INC.append(INC)
            self.IEN.append(IEN)
            #
            fNo = fNo + 1
            file_exists = os.path.exists(file_ID+str(fNo))
        # Number of Patches:
        self.np = fNo-1
        # ID ARRAY
        count = 0
        for i in range(0,self.np):
            nnp_p = self.nnp[i]
            ID_p = np.zeros((nnp_p,ndof)).astype(int)
            for j in range(0,nnp_p):
                for k in range(0,ndof):
                    ID_p[j,k] = count
                    count += 1
            self.ID.append(ID_p)
        # LM ARRAY
        for i in range(0,self.np):
            LM_p = np.zeros(((self.nel[i],self.nen[i],ndof))).astype(int)
            for j in range(0,self.nel[i]):
                for k in range(0,self.nen[i]):
                    LM_p[j,k,:] = self.ID[i][self.IEN[i][j,k],:]
            self.LM.append(LM_p)
    ############ 
    # SPAN INDEX
    def findSpan(self,pN,pD,u):
        if (u == self.U[pN][pD][self.nsf[pN][pD]+1]):
            return n
        low = self.p[pN][pD]
        high = self.nsf[pN][pD]+1
        mid = int((low+high)/2)
        while (u<self.U[pN][pD][mid] or u>self.U[pN][pD][mid+1]):
            if (u<self.U[pN][pD][mid]):
                high = mid
            else:
                low = mid
            mid = int(np.floor((low+high)/2))
        return mid
    ############
    # DERIVATIVES
    def dersBasisFuns(self,pN,pD,u,n):
        i = self.findSpan(pN,pD,u)
        left = np.zeros(self.p[pN][pD]+1)
        right = np.zeros(self.p[pN][pD]+1)
        ndu = np.zeros((self.p[pN][pD]+1,self.p[pN][pD]+1))
        a = np.zeros((2,self.p[pN][pD]+1))
        ders = np.zeros((n+1,self.p[pN][pD]+1))
        #
        ndu[0][0] = 1.0
        for j in range(1,self.p[pN][pD]+1):
            left[j] = u-self.U[pN][pD][i+1-j]
            right[j] = self.U[pN][pD][i+j]-u
            saved = 0.0
            for r in range(0,j):
                ndu[j][r] = right[r+1]+left[j-r]
                temp = ndu[r][j-1]/ndu[j][r]
                ndu[r][j] = saved+right[r+1]*temp
                saved = left[j-r]*temp
            ndu[j][j] = saved
        for j in range(0,self.p[pN][pD]+1):
            ders[0][j] = ndu[j][self.p[pN][pD]]
        for r in range(0,self.p[pN][pD]+1):
            s1 = 0
            s2 = 1
            a[0][0] = 1.0
            for k in range(1,n+1):
                d =0.0
                rk = r-k
                pk = self.p[pN][pD]-k
                if (r >= k):
                    a[s2][0] = a[s1][0]/ndu[pk+1][rk]
                    d = a[s2][0]*ndu[rk][pk]
                if (rk >= -1):
                    j1 = 1
                else:
                    j1 = -rk
                if (r-1 <= pk):
                    j2 = k-1
                else:
                    j2 = self.p[pN][pD]-r
                for j in range(j1,j2+1):
                    a[s2][j] = (a[s1][j]-a[s1][j-1])/ndu[pk+1][rk+j]
                    d += a[s2][j]*ndu[rk+j][pk]
                if (r <= pk):
                    a[s2][k] = -a[s1][k-1]/ndu[pk+1][r]
                    d += a[s2][k]*ndu[r][pk]
                ders[k][r] = d
                j = s1
                s1 = s2
                s2 = j
        r = self.p[pN][pD]
        for k in range(1,n+1):
            for j in range(0,self.p[pN][pD]+1):
                ders[k][j] *= r
            r *= (self.p[pN][pD]-k)
        return ders
    
    def getCP(self,pN,p_el):
        cp = np.zeros(((3,self.p[pN][0]+1,self.p[pN][1]+1)))
        wMat = np.zeros((self.p[pN][0]+1,self.p[pN][1]+1))
        iu = self.INC[pN][self.IEN[pN][p_el][0],0]
        iv = self.INC[pN][self.IEN[pN][p_el][0],1]
        ny = iv-self.p[pN][1]
        k = 0
        for j in range(0,self.p[pN][1]+1):
            nx = iu-self.p[pN][0]
            for i in range(0,self.p[pN][0]+1):
                cp[0][i,j] = self.CP[pN][self.nsf[pN][0]*ny+nx][0]
                cp[1][i,j] = self.CP[pN][self.nsf[pN][0]*ny+nx][1]
                cp[2][i,j] = self.CP[pN][self.nsf[pN][0]*ny+nx][2]
                wMat[i,j] = self.CP[pN][self.nsf[pN][0]*ny+nx][3]
                nx += 1
                k += 1
            ny += 1
        return cp, wMat
    
    def ratBasisFuns2D(self,pN,u,v,wMat,du,dv):
        N = self.dersBasisFuns(pN,0,u,du)
        M = self.dersBasisFuns(pN,1,v,dv)
        R = np.zeros((((self.p[pN][0]+1,self.p[pN][1]+1,du+1,dv+1))))
        wders = np.zeros((du+1,dv+1))
        for k in range(0,du+1):
            for l in range(0,dv+1):
                wders[k][l] = N[k][:].dot(wMat.dot(M[l][:]))
                temp1 = np.tensordot(N[k][:],(M[l][:]),0)*wMat
                for j in range(1,l+1):
                    temp1 -= (self.nchoosek(l,j)*wders[0][j])*R[:,:,k,l-j]
                for i in range(1,k+1):
                    temp1 -= (self.nchoosek(k,i)*wders[i][0])*R[:,:,k-i,l]
                    temp2 = np.zeros((self.p[pN][0]+1,self.p[pN][1]+1))
                    for j in range (1,l+1):
                        temp2 += (self.nchoosek(l,j)*wders[i][j])*R[:,:,k-i,l-j]
                    temp1 -= self.nchoosek(k,i)*temp2
                R[:,:,k,l] = temp1/wders[0][0]
        return R
    
    def nchoosek(self,n,r):
        return math.factorial(n)/math.factorial(r)/math.factorial(n-r)
    
    def surfaceDerivs(self,R,cp,du,dv):
        S = np.zeros(((3,du+1,dv+1)))
        for i in range(0,du+1):
            for j in range(0,dv+1):
                S[0][i,j] = np.tensordot(cp[0],R[:,:,i,j])
                S[1][i,j] = np.tensordot(cp[1],R[:,:,i,j])
                S[2][i,j] = np.tensordot(cp[2],R[:,:,i,j])
        return S

In [37]:
pf = patchFamily2D('igabem_DR05_4th_768_',2)

In [38]:
# Number of Patches
pf.np

3

In [39]:
# Knot Vectors:
pf.U

[[array([0.        , 0.        , 0.        , 0.        , 0.        ,
         0.04166667, 0.08333333, 0.125     , 0.16666667, 0.20833333,
         0.25      , 0.29166667, 0.33333333, 0.375     , 0.41666667,
         0.45833333, 0.5       , 0.54166667, 0.58333333, 0.625     ,
         0.66666667, 0.70833333, 0.75      , 0.79166667, 0.83333333,
         0.875     , 0.91666667, 0.95833333, 1.        , 1.        ,
         1.        , 1.        , 1.        ]),
  array([0.        , 0.        , 0.        , 0.        , 0.        ,
         0.02083333, 0.04166667, 0.0625    , 0.08333333, 0.10416667,
         0.125     , 0.14583333, 0.16666667, 0.1875    , 0.20833333,
         0.22916667, 0.25      , 0.27083333, 0.29166667, 0.3125    ,
         0.33333333, 0.35416667, 0.375     , 0.39583333, 0.41666667,
         0.4375    , 0.45833333, 0.47916667, 0.5       , 0.52083333,
         0.54166667, 0.5625    , 0.58333333, 0.60416667, 0.625     ,
         0.64583333, 0.66666667, 0.6875    , 0.70833333,

In [40]:
# Control Points:
pf.CP[0]

array([[-0.642     ,  0.        , -0.18      ,  1.        ],
       [-0.642     ,  0.00588556, -0.17997919,  1.        ],
       [-0.642     ,  0.01765466, -0.17939616,  1.        ],
       ...,
       [ 0.642     ,  0.01765464,  0.17939603,  1.        ],
       [ 0.642     ,  0.00588556,  0.17997917,  1.        ],
       [ 0.642     ,  0.        ,  0.18      ,  1.        ]])

In [41]:
# Order:
pf.p[0]

array([4, 4])

In [42]:
# Number of Shape Functions:
pf.nsf[0]

array([28, 52])

In [43]:
pf.INC[0]

array([[ 0,  0],
       [ 1,  0],
       [ 2,  0],
       ...,
       [25, 51],
       [26, 51],
       [27, 51]])

In [44]:
N = pf.dersBasisFuns(0,0,0.025,2)
M = pf.dersBasisFuns(0,1,0.025,2)
print(N)
print(M)

[[ 2.56000123e-02  4.28999994e-01  4.40999997e-01  9.89999974e-02
   5.39999968e-03]
 [-6.14400172e+00 -2.06399987e+01  1.58400006e+01  1.00799998e+01
   8.63999948e-01]
 [ 1.10592009e+03 -5.18400162e+02 -1.20959993e+03  5.18400006e+02
   1.03679994e+02]]
[[ 5.12000123e-02  4.29599985e-01  4.33066660e-01  8.60666761e-02
   6.66668480e-05]
 [-1.22880005e+01 -2.53439976e+01  2.40639970e+01  1.35040010e+01
   6.40001229e-02]
 [ 2.21183965e+03 -1.65887960e+03 -2.02751999e+03  1.42847990e+03
   4.60800516e+01]]


In [45]:
pf.IEN[0]

array([[ 116,  115,  114, ...,    2,    1,    0],
       [ 117,  116,  115, ...,    3,    2,    1],
       [ 118,  117,  116, ...,    4,    3,    2],
       ...,
       [1453, 1452, 1451, ..., 1339, 1338, 1337],
       [1454, 1453, 1452, ..., 1340, 1339, 1338],
       [1455, 1454, 1453, ..., 1341, 1340, 1339]])

In [46]:
pf.ID[0]

array([[   0,    1],
       [   2,    3],
       [   4,    5],
       ...,
       [2906, 2907],
       [2908, 2909],
       [2910, 2911]])

In [47]:
cp, wMat = pf.getCP(0,0)
wMat

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [36]:
pf.LM[0][199]

array([[526, 527],
       [524, 525],
       [522, 523],
       [482, 483],
       [480, 481],
       [478, 479],
       [438, 439],
       [436, 437],
       [434, 435]])

In [141]:
cp

array([[[0.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25],
        [0.75, 0.75, 0.75]],

       [[0.  , 0.25, 0.75],
        [0.  , 0.25, 0.75],
        [0.  , 0.25, 0.75]],

       [[0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  ]]])

In [150]:
R = pf.ratBasisFuns2D(0,0.025,0.025,wMat,2,2)
R[:,:,0,2]

array([[  50. ,  -75. ,   25. ],
       [ 125. , -187.5,   62.5],
       [  25. ,  -37.5,   12.5]])

In [156]:
S = pf.surfaceDerivs(R,cp,2,2)
S[:,0,1]

array([-4.4408921e-16,  5.0000000e+00,  0.0000000e+00])

In [138]:
S[:][0,0]

array([0.25, 0.  , 0.  ])